<a href="https://colab.research.google.com/github/kocherlab/naps/blob/main/docs/notebooks/naps_basic_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NAPS Basic Workflow

In this notebook we'll install NAPS, pull the example data from the [GitHub repository](https://github.com/kocherlab/naps), and run `naps-track` against it.

This repo is particularly useful in combination with SLEAP's example notebook on remote training and inference which can be found [here](https://colab.research.google.com/github/talmolab/sleap/blob/main/docs/notebooks/Training_and_inference_on_an_example_dataset.ipynb).

## Install NAPS

In [ ]:
!pip install -q naps-track

## Download sample training data into Colab
Let's download a sample dataset from the the NAPS repository.


In [ ]:
!wget https://github.com/kocherlab/naps/raw/main/tests/data/example.slp
!wget https://github.com/kocherlab/naps/raw/main/tests/data/example.analysis.h5
!wget https://github.com/kocherlab/naps/raw/main/tests/data/example.mp4
!ls -lht

## NAPS tracking
Now let's track the files using `naps-track`.

In [ ]:
!naps-track --slp-path example.slp --h5-path example.analysis.h5 --video-path example.mp4 --tag-node 0 --start-frame 0 --end-frame 1203 --aruco-marker-set DICT_4X4_100 --output-path example_output.analysis.h5

In [ ]:
import h5py
import numpy as np

filename = "example_output.analysis.h5"

with h5py.File(filename, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

print("===filename===")
print(filename)
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

## Download

Now we can just download the output!

In [ ]:
# Download.
from google.colab import files

files.download("/content/example_output.analysis.h5")

If you happen to not be using Chrome, you may get an error here. If that happens, you should be able to download the files using the "Files" tab on the left side panel.